# Imports

In [55]:
import pandas as pd
import os
import string
from itertools import groupby
from heapq import nlargest
import re
import numpy as np

In [2]:
def open_source(day):
    with open(f'source/day{day}.txt', 'r') as f:
        lista = f.readlines()
    lista = [x.strip('\n') for x in lista]
    return lista

In [3]:
path = '/Users/barnabas.kadar/Own Repos/advent/2022/source'

def create_txt_files(path):
    files = [f"day{x}" if x > 9 else f"day0{x}" for x in range(1, 26)]
    for file in files:
        try:
            with open(os.path.join(path, file + ".txt"), 'x') as fp:
                pass
            with open(os.path.join(path, file + "_example.txt"), 'x') as fp:
                pass
        except FileExistsError:
            print("file existed but continued loop")
            continue

# Day 1

In [171]:
lista = open_source("01")

In [183]:
result = [sum([int(x) for x in a[1]]) for a in groupby(lista, key=lambda s: not s) if not a[0]]

In [184]:
max(result)

71780

In [187]:
sum(nlargest(3, result))

212489

# Day 2

In [71]:
lista = open_source("02")

In [72]:
scoring1 = {
    'A X': 4,
    'A Y': 8,
    'A Z': 3,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 7,
    'C Y': 2,
    'C Z': 6
}

In [73]:
scoring2 = {
    'A X': 3,
    'A Y': 4,
    'A Z': 8,
    'B X': 1,
    'B Y': 5,
    'B Z': 9,
    'C X': 2,
    'C Y': 6,
    'C Z': 7
}

In [74]:
sum([scoring1[x] for x in lista])

10624

In [75]:
sum([scoring2[x] for x in lista])

14060

# Day 3

In [4]:
lista = open_source("03")

In [5]:
scoring = dict(zip(list(string.ascii_lowercase) + list(string.ascii_uppercase), list(range(1, 53))))

In [26]:
total = 0
for sack in lista:
    half = len(sack)//2
    first, second = set(sack[half:]), set(sack[:half])
    inter = set.intersection(first, second).pop()
    total += scoring[inter]
total

7821

In [7]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [25]:
total = 0
for sack in chunks(lista, 3):
    inter = set.intersection(*list(map(lambda x: set(x), sack)))
    total += scoring[inter.pop()]
total

2752

# Day 4

In [5]:
lista = open_source('04')

In [10]:
count1 = count2 = 0
for task in lista:
    fs, fe, ss, se = list(map(int, re.split(',|-', task)))
    if (fs >= ss and fe <= se) or (fs <= ss and fe >= se):
        count1 += 1
    if fs <= se and ss <= fe:
        count2 += 1
print(count1, count2)

513 878


# Day 5

In [224]:
lista = open_source('05_example')

In [214]:
def get_day5_part1(s_dict, proc):
    _stack_dict = s_dict.copy()
    for c in proc:
        for times in range(c[0]):
            moved_item = _stack_dict[c[1]].pop(-1)
            _stack_dict[c[2]].append(moved_item)
    result = ''
    for k, v in _stack_dict.items():
        result += v[-1]
    return result

In [215]:
def get_day5_part2(s_dict, proc):
    _stack_dict = s_dict.copy()
    for c in proc:
        length = c[0]
        moved_item = _stack_dict[c[1]][-length:]
        del _stack_dict[c[1]][-length:]
        _stack_dict[c[2]] = _stack_dict[c[2]] + moved_item
    result = ''
    for k, v in _stack_dict.items():
        result += v[-1]
    return result

In [213]:
def convert_day5_input(lista):
    stacks = []
    proc = []
    for sor in lista:
        if '[' in sor:
            sor = [x.lstrip('[').rstrip(']') for x in re.split('    |\] \[| \[', sor)]
            stacks.append(sor)
        elif ' 1   2   3 ' in sor:
            sor = [x for x in sor.split(' ') if x]
            stacks.append(sor)
        elif 'move' in sor:
            sor = [int(x) for x in sor.split(' ') if x.isdigit()]
            proc.append(sor)
    stacks = np.rot90(stacks, 3)
    stack_dict = {}
    for stack in stacks:
        k, *v = [x for x in stack if x]
        stack_dict[int(k)] = v
    return stack_dict, proc

In [225]:
lista

['    [D]    ',
 '[N] [C]    ',
 '[Z] [M] [P]',
 ' 1   2   3 ',
 '',
 'move 1 from 2 to 1',
 'move 3 from 1 to 3',
 'move 2 from 2 to 1',
 'move 1 from 1 to 2']

In [228]:
for sor in lista:
    if 'move' not in sor:
        print(sor[1::4])

 D 
NC 
ZMP
123



In [220]:
s, p = convert_day5_input(lista)
s

{1: ['P', 'F', 'M', 'Q', 'W', 'G', 'R', 'T'],
 2: ['H', 'F', 'R'],
 3: ['P', 'Z', 'R', 'V', 'G', 'H', 'S', 'D'],
 4: ['Q', 'H', 'P', 'B', 'F', 'W', 'G'],
 5: ['P', 'S', 'M', 'J', 'H'],
 6: ['M', 'Z', 'T', 'H', 'S', 'R', 'P', 'L'],
 7: ['P', 'T', 'H', 'N', 'M', 'L'],
 8: ['F', 'D', 'Q', 'R'],
 9: ['D', 'S', 'C', 'N', 'L', 'P', 'H']}

In [222]:
p[:5]

[[3, 8, 9], [2, 2, 8], [5, 4, 2], [7, 1, 4], [3, 8, 2]]

In [216]:
get_day5_part1(s, p)

'TPGVQPFDH'

In [217]:
get_day5_part2(s, p)

'NQGMFRWMP'

In [212]:
s, p = convert_day5_input(lista)
get_day5_part2(s, p)

'DMRDFRHHH'

In [219]:
print("".join(z[-1] for z in stacks[1:]))



stacks = [
    [],
    list("QSWCZVFT"),
    list("QRB"),
    list("BZTQPMS"),
    list("DVFRQH"),
    list("JGLDBSTP"),
    list("WRTZ"),
    list("HQMNSFRJ"),
    list("RNFHW"),
    list("JZTQPRB")
]


for l in s[10:]:
    l = l[5:]
    a,r = l.split(" from ")
    b,c = r.split(" to ")
    a=int(a)
    b=int(b)
    c=int(c)
    stacks[c].extend(stacks[b][-a:])
    del stacks[b][-a:]
#    for _ in range(a):
#        stacks[b].pop(-1)


print("".join(z[-1] for z in stacks[1:]))

TBSHPZJWB


TypeError: unhashable type: 'slice'

In [205]:
get_day5_part2(s, p)

IndexError: list index out of range

In [181]:
for c in proc:
    length = c[0]
    moved_item = stack_dict[c[1]][-length:]
    del stack_dict[c[1]][-length:]
    stack_dict[c[2]] = stack_dict[c[2]] + moved_item
result = ''
for k, v in stack_dict.items():
    result += v[-1]
result

'DMRDFRHHH'

# Day 6

# Day 7

# Day 8

# Day 9

# Day 10

# Day 11

# Day 12

# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25